# GROUPING

After preparing the data for a more exhaustive analysis, the last step is to separate the different countries into group with common characteristics. This is necessary as an aggregate study of all countries would mean erasing the differences between them and missing important conclusions, making our study too shallow.

In [28]:
import os
import pandas as pd
column_country = 'Country'
column_region = 'Region'

read_path = os.getcwd()
region_url = read_path + '/world-regions-according-to-the-world-bank.csv'
df_url = read_path + '/Output/SilverDataframe.csv'

In [39]:
region_df = pd.read_csv(region_url)
df = pd.read_csv(df_url)

print(region_df)
region_df.rename(columns = {'World Region according to the World Bank': column_region, 'Entity': column_country}, inplace = True)
region_df.drop(columns=['Code', 'Year'], inplace = True)
print(region_df)

print(df)
df = pd.merge(df, region_df)
print(df)

for region in set(region_df[column_region].to_list()):
    df.loc[df[column_region] == region].to_csv(read_path + '/Output/GoldDataframe_' + region + '.csv')

C:\Users\vperezlo\AppData\Local\Temp\ipykernel_2380\1068467184.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_url)


             Entity Code  Year World Region according to the World Bank
0       Afghanistan  AFG  2017                               South Asia
1           Albania  ALB  2017                  Europe and Central Asia
2           Algeria  DZA  2017             Middle East and North Africa
3    American Samoa  ASM  2017                    East Asia and Pacific
4           Andorra  AND  2017                  Europe and Central Asia
..              ...  ...   ...                                      ...
212       Venezuela  VEN  2017              Latin America and Caribbean
213         Vietnam  VNM  2017                    East Asia and Pacific
214           Yemen  YEM  2017             Middle East and North Africa
215          Zambia  ZMB  2017                       Sub-Saharan Africa
216        Zimbabwe  ZWE  2017                       Sub-Saharan Africa

[217 rows x 4 columns]
            Country                        Region
0       Afghanistan                    South Asia
1           

In [51]:
df.loc[:, 1:2]

TypeError: cannot do slice indexing on Index with these indexers [1] of type int

In [ ]:
# K-MEANS CLUSTERING
# Importing Modules
from sklearn import datasets
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
# Loading dataset
iris_df = datasets.load_iris()

# Declaring Model
model = KMeans(n_clusters=3)

# Fitting Model
model.fit(iris_df.data)

# Predicitng a single input
predicted_label = model.predict([[7.2, 3.5, 0.8, 1.6]])

# Prediction on the entire data
all_predictions = model.predict(iris_df.data)

# Printing Predictions
print(predicted_label)
print(all_predictions)


# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :3]  # we only take the first two features.
y = iris.target


fig = plt.figure(figsize=(10,10))
plt = fig.add_subplot(111, projection='3d')
plt.scatter(X[:,0],X[:,1],X[:,2], 
            c=all_predictions, edgecolor='red', s=40, alpha = 0.5)
plt.set_title("First three PCA directions")
plt.set_xlabel("Educational_Degree")
plt.set_ylabel("Gross_Monthly_Salary")
plt.set_zlabel("Claim_Rate")
plt.dist = 10
plt

In [42]:
from sklearn import datasets
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [47]:
model = KMeans()
res = model.fit_predict(df)

ValueError: could not convert string to float: 'Afghanistan'